In [55]:
# import modules
import glob
import pandas as pd
import geopandas as gpd
import numpy as np

In [56]:
# read travel times
ttimes = gpd.read_file('results/travel_times_raw.shp')
ttimes.head(2)

,x,y,YKR_ID,pt_r_t_0,pt_r_t_1,pt_r_t_2,pt_r_t_3,pt_r_t_4,pt_r_t_5,geometry
0,381875.0,6697880.0,5785640,132,109,141,109,101,124,"POLYGON ((382000.0001388059 6697750.000128186,..."
1,382125.0,6697880.0,5785641,135,112,143,112,108,127,"POLYGON ((382250.00013875 6697750.000128181, 3..."


In [57]:
# replace -1 with No Data
ttimes = ttimes.replace(-1, np.nan)

In [58]:
# get time column names as list
timecols = []
for col in list(ttimes):
    if 'pt_r_t_' in col:
        timecols.append(col)

In [59]:
# calculate minimum travel time to column 'min_t'
ttimes['min_t'] = ttimes.apply(lambda row: row[timecols].min(), axis=1)

ttimes.head(2)

,x,y,YKR_ID,pt_r_t_0,pt_r_t_1,pt_r_t_2,pt_r_t_3,pt_r_t_4,pt_r_t_5,geometry,min_t
0,381875.0,6697880.0,5785640,132.0,109.0,141.0,109.0,101.0,124.0,"POLYGON ((382000.0001388059 6697750.000128186,...",101.0
1,382125.0,6697880.0,5785641,135.0,112.0,143.0,112.0,108.0,127.0,"POLYGON ((382250.00013875 6697750.000128181, 3...",108.0


In [60]:
# add index of the smallest travel time to the df
ttimes['min_idx'] = ttimes.apply(lambda row: row[timecols].astype(float).idxmin(), axis=1)

ttimes.head(2)

,x,y,YKR_ID,pt_r_t_0,pt_r_t_1,pt_r_t_2,pt_r_t_3,pt_r_t_4,pt_r_t_5,geometry,min_t,min_idx
0,381875.0,6697880.0,5785640,132.0,109.0,141.0,109.0,101.0,124.0,"POLYGON ((382000.0001388059 6697750.000128186,...",101.0,pt_r_t_4
1,382125.0,6697880.0,5785641,135.0,112.0,143.0,112.0,108.0,127.0,"POLYGON ((382250.00013875 6697750.000128181, 3...",108.0,pt_r_t_4


In [61]:
# drop rows having nan values in 'min_t' or 'min_idx' columns
count_withnan = len(ttimes)
ttimes = ttimes.dropna(subset=['min_t', 'min_idx'])
count_withoutnan = len(ttimes)

print('dropped', count_withnan - count_withoutnan, 'rows with na values')

dropped 211 rows with na values


In [62]:
ttimes.head(2)

,x,y,YKR_ID,pt_r_t_0,pt_r_t_1,pt_r_t_2,pt_r_t_3,pt_r_t_4,pt_r_t_5,geometry,min_t,min_idx
0,381875.0,6697880.0,5785640,132.0,109.0,141.0,109.0,101.0,124.0,"POLYGON ((382000.0001388059 6697750.000128186,...",101.0,pt_r_t_4
1,382125.0,6697880.0,5785641,135.0,112.0,143.0,112.0,108.0,127.0,"POLYGON ((382250.00013875 6697750.000128181, 3...",108.0,pt_r_t_4


In [63]:
# float to int
ttimes['min_t'] = [int(value) for value in ttimes['min_t']]

# save min travel times to shp
outfp = 'results/travel_times_min.shp'
ttimes.to_file(outfp)